In [ ]:
!pip install albumentations > /dev/null
!pip install -U efficientnet==0.0.4
%tensorflow_version 1.x
import numpy as np
import pandas as pd
import gc
import keras

import random
import cv2

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split,StratifiedKFold

from skimage.transform import resize
import tensorflow as tf
import keras.backend as K
from keras.losses import binary_crossentropy

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import  ModelCheckpoint
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras import backend as K
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.losses import binary_crossentropy
import keras.callbacks as callbacks
from keras.callbacks import Callback
from keras.applications.xception import Xception
from keras.layers import multiply


from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects

from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.regularizers import l2
from keras.layers.core import Dense, Lambda
from keras.layers.merge import concatenate, add
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Permute
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

import glob
import shutil
import os
import random
from PIL import Image

seed = 10
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

tf.set_random_seed(seed)
    
%matplotlib inline

  Created wheel for efficientnet: filename=efficientnet-0.0.4-cp36-none-any.whl size=14288 sha256=b84dcb45f5b29ce250aead3bbbb828644e9cc076c6f2230b613fe64493b8a293
  Stored in directory: /root/.cache/pip/wheels/5c/34/68/a611a699a28239e964ccf144c0e767cdb5439fee82ec5de6e0
Successfully built efficientnet
TensorFlow is already loaded. Please restart the runtime to change versions.


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


AttributeError: ignored

In [ ]:
a = []
while (True):
  a.append('1')

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [ ]:
def get_iou_vector(A, B):
    # Numpy version    
    batch_size = A.shape[0]
    metric = 0.0
    for batch in range(batch_size):
        t, p = A[batch], B[batch]
        true = np.sum(t)
        pred = np.sum(p)
        
        # deal with empty mask first
        if true == 0:
            metric += (pred == 0)
            continue
        
        # non empty mask case.  Union is never empty 
        # hence it is safe to divide by its number of pixels
        intersection = np.sum(t * p)
        union = true + pred - intersection
        iou = intersection / union
        
        # iou metrric is a stepwise approximation of the real iou over 0.5
        iou = np.floor(max(0, (iou - 0.45)*20)) / 10
        
        metric += iou
        
    # teake the average over all images in batch
    metric /= batch_size
    return metric


def my_iou_metric(label, pred):
    # Tensorflow version
    return tf.py_func(get_iou_vector, [label, pred > 0.5], tf.float64)

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def bce_logdice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))

In [ ]:
class SnapshotCallbackBuilder:
    def __init__(self, nb_epochs, nb_snapshots, init_lr=0.1):
        self.T = nb_epochs
        self.M = nb_snapshots
        self.alpha_zero = init_lr

    def get_callbacks(self, model_prefix='Model'):

        callback_list = [
            callbacks.ModelCheckpoint("./keras.model",monitor='val_my_iou_metric', 
                                   mode = 'max', save_best_only=True, verbose=1),
            swa,
            callbacks.LearningRateScheduler(schedule=self._cosine_anneal_schedule)
        ]

        return callback_list

    def _cosine_anneal_schedule(self, t):
        cos_inner = np.pi * (t % (self.T // self.M))  # t - 1 is used when t has 1-based indexing.
        cos_inner /= self.T // self.M
        cos_out = np.cos(cos_inner) + 1
        return float(self.alpha_zero / 2 * cos_out)

In [ ]:
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation == True:
        x = LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = LeakyReLU(alpha=0.1)(blockInput)
    x = BatchNormalization()(x)
    blockInput = BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    return x

In [ ]:
from efficientnet import EfficientNetB4

def UEfficientNet(input_shape=(None, None, 1),dropout_rate=0.1):

    backbone = EfficientNetB4(weights='imagenet',
                            include_top=False,
                            input_shape=input_shape)
    input = backbone.input
    start_neurons = 8

    conv4 = backbone.layers[342].output
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)
    
     # Middle
    convm = Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same",name='conv_middle')(pool4)
    convm = residual_block(convm,start_neurons * 32)
    convm = residual_block(convm,start_neurons * 32)
    convm = LeakyReLU(alpha=0.1)(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    deconv4_up1 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4)
    deconv4_up2 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4_up1)
    deconv4_up3 = Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(deconv4_up2)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(dropout_rate, name='conv_mid_right')(uconv4) 
    
    uconv4 = Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
#   uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = LeakyReLU(alpha=0.1)(uconv4)  #conv1_2
    
    deconv3 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
    deconv3_up1 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(deconv3)
    deconv3_up2 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(deconv3_up1)
    conv3 = backbone.layers[154].output
    uconv3 = concatenate([deconv3,deconv4_up1, conv3])    
    uconv3 = Dropout(dropout_rate)(uconv3)
    
    uconv3 = Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 8)
#     uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = LeakyReLU(alpha=0.1)(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
    deconv2_up1 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(deconv2)
    conv2 = backbone.layers[92].output
    uconv2 = concatenate([deconv2,deconv3_up1,deconv4_up2, conv2])
        
    uconv2 = Dropout(0.1)(uconv2)
    uconv2 = Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 4)
#     uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = LeakyReLU(alpha=0.1)(uconv2)
    
    deconv1 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    conv1 = backbone.layers[30].output
    uconv1 = concatenate([deconv1,deconv2_up1,deconv3_up2,deconv4_up3, conv1])
    
    uconv1 = Dropout(0.1)(uconv1)
    uconv1 = Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 2)
#     uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = LeakyReLU(alpha=0.1)(uconv1)
    
    uconv0 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)   
    uconv0 = Dropout(0.1)(uconv0)
    uconv0 = Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
    uconv0 = residual_block(uconv0,start_neurons * 1)
#     uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = LeakyReLU(alpha=0.1)(uconv0)
    
    uconv0 = Dropout(dropout_rate/2)(uconv0)
    output_layer = Conv2D(3, (1,1), padding="same", activation="softmax")(uconv0)    
    
    model = Model(input, output_layer)
    model.name = 'u-xception'

    return model

In [ ]:
K.clear_session()
img_size = 256
model = UEfficientNet(input_shape=(img_size,img_size,3),dropout_rate=0.5)

AttributeError: ignored

In [ ]:
model.summary()

Model: "u-xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 128, 128, 48) 0           batch_normalization_1[0][0]      
_________________________________________________________________________________________

In [ ]:
u_net_weights = "/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/final_unet/unet++efficientnet/full_coronacases_unet++efficientnet.h5"
model.load_weights(u_net_weights)

In [ ]:
for layer in model.layers:
	layer.trainable = False


In [ ]:
out = model.get_layer("conv_middle").output
model_cut_en = Model(inputs = model.input, outputs = out)
model_cut_en.summary()

out = model.get_layer("conv_mid_right").output
model_cut_right = Model(inputs = model.input, outputs = out)
#model_cut_right.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 128, 128, 48) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [ ]:
def volume_feature_map(in_shape):
    x_in_0 = Input(in_shape)
    slice_num= x_in_0.shape[-2]
    print(x_in_0.shape)
    x_in = Reshape((256, 256,slice_num, 3))(x_in_0)
    branch = []
    for i in range(slice_num):
        x_out = Lambda(lambda x: x[:,:,:,i,:])(x_in)
        x_out = model_cut_en(x_out)
        #print(x_out.shape)
        x_out = Conv2D(32, (3, 3), activation='relu', padding='same')(x_out)
        #print(x_out.shape)
        branch.append(x_out)
        print(i)
        
    
    x = Concatenate(axis=-1)(branch)
    
    
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    print(x.shape)
    x = Dense(128, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    model = Model(x_in_0, x)
    
    
    return model

In [ ]:
def classifier(x, model, mode=0):
    
    def just_conv(new_in, ker):
        new_in =  Conv2D(new_in.shape[3], (ker, ker), activation='relu', padding='same')(new_in)
        new_in = BatchNormalization()(new_in)
        return new_in
    
    def jump_add(new_in, mode= mode):
        if mode==0:
          new_in_1 = conv_max(new_in, 3)
        
        elif mode==1:
          new_in_1 = just_conv(new_in, 3)
          new_in_2 = just_conv(new_in_1, 3)
          new_in_2 = just_conv(new_in_2, 3)
            
          new_in_1 = Add()([new_in_1, new_in_2])

        elif mode==2:
          new_in_1 = conv_max(new_in, 3)
          new_in_2 = just_conv(new_in_1, 3)
          new_in_2 = just_conv(new_in_2, 3)

          new_in_3 = just_conv(new_in_1, 1)
          new_in_3 = just_conv(new_in_3, 1)
          new_in_1 = Add()([new_in_1, new_in_2, new_in_3])

        return new_in_1


In [ ]:
model_appended=volume_feature_map((256, 256, 5, 3)) #change 10 to 35 or 40

(?, 256, 256, 5, 3)
0
1
2
3
4
(?, 256)


In [ ]:
model_appended.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 5,  0                                            
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 256, 256, Dim 0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_38 (Lambda)              (None, 256, 256, 3)  0           reshape_2[0][0]                  
__________________________________________________________________________________________________
lambda_39 (Lambda)              (None, 256, 256, 3)  0           reshape_2[0][0]                  
____________________________________________________________________________________________

In [ ]:
im_dir = "/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/CLASSIFIER_PATIENT_WISE/train/"
a1 = os.path.join(im_dir, os.listdir(im_dir)[0])
a2 = os.path.join(im_dir, os.listdir(im_dir)[1])
bunches = 50
patients = len(os.listdir(a1))+len(os.listdir(a2))



9

In [ ]:
init = 0  #don't exceed 5
bunch_1= bunches/2
bunch_0= bunches/2

if (len(os.listdir(a1))-bunch_1*(init+1)) <10: 
  rest1 = int(len(os.listdir(a1))-bunch_1*(init+1))
else :
  rest1 = 0

if (len(os.listdir(a2))-bunch_0*(init+1)) <10: 
  rest2 = int(len(os.listdir(a2))-bunch_0*(init+1))
else :
  rest2 = 0

X = np.empty((bunches+rest1+rest2, 256, 256, 29, 3), dtype= np.float64)
Y = np.empty((bunches+rest1+rest2), dtype= int)

X.shape

(50, 256, 256, 29, 3)

In [ ]:

for j in range(int(bunch_1*init), int(bunch_1*(init+1)+rest1)):
    a1_1= os.path.join(a1, os.listdir(a1)[j])  #patients
    a1_1_1= np.sort(random.sample(os.listdir(a1_1), 29))  #29 sampling
    for m in range(len(a1_1_1)):
        a111_im = os.path.join(a1_1, a1_1_1[m])
        a111_im = cv2.imread(a111_im)
        a111_im = resize(a111_im, (256, 256), anti_aliasing=True)
        X[j, :,:,m,: ] = a111_im
        print("patient : " + str(j) + " slice : " +str(m))
        Y[j] = 1

patient : 0 slice : 0
patient : 0 slice : 1
patient : 0 slice : 2
patient : 0 slice : 3
patient : 0 slice : 4
patient : 0 slice : 5
patient : 0 slice : 6
patient : 0 slice : 7
patient : 0 slice : 8
patient : 0 slice : 9
patient : 0 slice : 10
patient : 0 slice : 11
patient : 0 slice : 12
patient : 0 slice : 13
patient : 0 slice : 14
patient : 0 slice : 15
patient : 0 slice : 16
patient : 0 slice : 17
patient : 0 slice : 18
patient : 0 slice : 19
patient : 0 slice : 20
patient : 0 slice : 21
patient : 0 slice : 22
patient : 0 slice : 23
patient : 0 slice : 24
patient : 0 slice : 25
patient : 0 slice : 26
patient : 0 slice : 27
patient : 0 slice : 28
patient : 1 slice : 0
patient : 1 slice : 1
patient : 1 slice : 2
patient : 1 slice : 3
patient : 1 slice : 4
patient : 1 slice : 5
patient : 1 slice : 6
patient : 1 slice : 7
patient : 1 slice : 8
patient : 1 slice : 9
patient : 1 slice : 10
patient : 1 slice : 11
patient : 1 slice : 12
patient : 1 slice : 13
patient : 1 slice : 14
patient 

In [ ]:
int(bunch_1*(init+1)+rest1)+int(bunch_0*(init+1)+rest2)

50

In [ ]:


for j in range(int(bunch_1*(init+1)+rest1)+int(bunch_0*init),int(bunch_1*(init+1)+rest1)+int(bunch_0*(init+1)+rest2)):
    a2_1= os.path.join(a2, os.listdir(a2)[j])  #patients
    a2_1_1= np.sort(random.sample(os.listdir(a2_1), 29))  #29 sampling
    for m in range(len(a2_1_1)):
        a211_im = os.path.join(a2_1, a2_1_1[m])
        a211_im = cv2.imread(a211_im)
        a211_im = resize(a211_im, (256, 256), anti_aliasing=True)
        X[j, :,:,m,: ] = a211_im
        print("patient : " + str(j) + " slice : " +str(m))
        Y[j] = 0

patient : 25 slice : 0
patient : 25 slice : 1
patient : 25 slice : 2
patient : 25 slice : 3
patient : 25 slice : 4
patient : 25 slice : 5
patient : 25 slice : 6
patient : 25 slice : 7
patient : 25 slice : 8
patient : 25 slice : 9
patient : 25 slice : 10
patient : 25 slice : 11
patient : 25 slice : 12
patient : 25 slice : 13
patient : 25 slice : 14
patient : 25 slice : 15
patient : 25 slice : 16
patient : 25 slice : 17
patient : 25 slice : 18
patient : 25 slice : 19
patient : 25 slice : 20
patient : 25 slice : 21
patient : 25 slice : 22
patient : 25 slice : 23
patient : 25 slice : 24
patient : 25 slice : 25
patient : 25 slice : 26
patient : 25 slice : 27
patient : 25 slice : 28
patient : 26 slice : 0
patient : 26 slice : 1
patient : 26 slice : 2
patient : 26 slice : 3
patient : 26 slice : 4
patient : 26 slice : 5
patient : 26 slice : 6
patient : 26 slice : 7
patient : 26 slice : 8
patient : 26 slice : 9
patient : 26 slice : 10
patient : 26 slice : 11
patient : 26 slice : 12
patient : 26

In [ ]:

a1_1= os.path.join(a1, os.listdir(a1)[4])
#len(os.listdir(a1))
a1_1

'/content/drive/My Drive/CT_SCAN_SARS-COV_2_datasets/dataset/CLASSIFIER_PATIENT_WISE/train/COVID/study_0948.nii.gz'